In [52]:
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from moviesMapping import moviesMapping
import pandas as pd
import json
import requests
import os

In [53]:
es = Elasticsearch("http://localhost:9200")
es.ping()

True

In [54]:
# Ruta al archivo JSON local
directorio_json = '/Users/varela/IA'

# Lista para almacenar los DataFrames individuales
dataframes = []

# Lista de archivos JSON en el directorio
archivos_json = [archivo for archivo in os.listdir(directorio_json) if archivo.endswith('.json')]

# Leer cada archivo JSON, cargarlo en un DataFrame y almacenarlo en la lista de DataFrames
for archivo_json in archivos_json:
    with open(os.path.join(directorio_json, archivo_json), 'r') as archivo:
        datos_json = json.load(archivo)
        df = pd.json_normalize(datos_json)  # Normalizar el JSON en un DataFrame
        dataframes.append(df)

# Combinar los DataFrames en uno solo
df = pd.concat(dataframes, ignore_index=True)

In [48]:
df.head()

,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height
0,The Adventures of Ford Fairlane,1990,"[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","[Action, Comedy, Mystery]",The_Adventures_of_Ford_Fairlane,The Adventures of Ford Fairlane is a 1990 Amer...,https://upload.wikimedia.org/wikipedia/en/a/ac...,259.0,384.0
1,"After Dark, My Sweet",1990,"[Jason Patric, Rachel Ward, Bruce Dern]","[Mystery, Crime, Noir, Thriller]","After_Dark,_My_Sweet","After Dark, My Sweet is a 1990 American neo-no...",https://upload.wikimedia.org/wikipedia/en/e/e2...,255.0,383.0
2,Air America,1990,"[Mel Gibson, Robert Downey, Jr., Nancy Travis]","[Action, Comedy]",Air_America_(film),Air America is a 1990 American action comedy f...,https://upload.wikimedia.org/wikipedia/en/1/11...,258.0,385.0
3,Alice,1990,"[Mia Farrow, Joe Mantegna, William Hurt, Alec ...","[Comedy, Fantasy, Romance]",Alice_(1990_film),Alice is a 1990 American fantasy romantic come...,https://upload.wikimedia.org/wikipedia/en/e/e4...,220.0,343.0
4,Almost an Angel,1990,"[Paul Hogan, Elias Koteas, Charlton Heston]","[Comedy, Drama, Fantasy]",Almost_an_Angel,Almost an Angel is a 1990 American fantasy com...,https://upload.wikimedia.org/wikipedia/en/f/f7...,261.0,381.0


In [55]:
#df.isna().value_counts()
print(df.isna().sum())

title                 0
year                  0
cast                  0
genres                0
href                152
extract             158
thumbnail           631
thumbnail_width     631
thumbnail_height    631
dtype: int64


In [56]:
df = df.dropna(subset=['extract'])
print(df.isna().sum())

title                 0
year                  0
cast                  0
genres                0
href                  0
extract               0
thumbnail           473
thumbnail_width     473
thumbnail_height    473
dtype: int64


In [57]:
len(df)


12675

In [58]:
df = df.drop_duplicates(subset=['title'])
len(df)

12212

In [59]:
model = SentenceTransformer('all-mpnet-base-v2')

In [60]:
df["vector"] = df["extract"].apply(lambda x: model.encode(x))

In [61]:
df.head()

,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height,vector
0,The Adventures of Ford Fairlane,1990,"[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","[Action, Comedy, Mystery]",The_Adventures_of_Ford_Fairlane,The Adventures of Ford Fairlane is a 1990 Amer...,https://upload.wikimedia.org/wikipedia/en/a/ac...,259.0,384.0,"[-0.016863724, 0.010160819, 0.018435642, 0.029..."
1,"After Dark, My Sweet",1990,"[Jason Patric, Rachel Ward, Bruce Dern]","[Mystery, Crime, Noir, Thriller]","After_Dark,_My_Sweet","After Dark, My Sweet is a 1990 American neo-no...",https://upload.wikimedia.org/wikipedia/en/e/e2...,255.0,383.0,"[0.027006403, 0.006256058, 0.014853085, -0.017..."
2,Air America,1990,"[Mel Gibson, Robert Downey, Jr., Nancy Travis]","[Action, Comedy]",Air_America_(film),Air America is a 1990 American action comedy f...,https://upload.wikimedia.org/wikipedia/en/1/11...,258.0,385.0,"[-0.0124979075, 0.07573932, 0.019687746, -0.01..."
3,Alice,1990,"[Mia Farrow, Joe Mantegna, William Hurt, Alec ...","[Comedy, Fantasy, Romance]",Alice_(1990_film),Alice is a 1990 American fantasy romantic come...,https://upload.wikimedia.org/wikipedia/en/e/e4...,220.0,343.0,"[0.04869419, 0.030663753, -0.0024175271, 0.024..."
4,Almost an Angel,1990,"[Paul Hogan, Elias Koteas, Charlton Heston]","[Comedy, Drama, Fantasy]",Almost_an_Angel,Almost an Angel is a 1990 American fantasy com...,https://upload.wikimedia.org/wikipedia/en/f/f7...,261.0,381.0,"[-0.0028605615, -0.025755227, 0.023163091, -0...."


In [62]:
es.indices.create(index="movies", mappings=moviesMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

In [63]:
record_list = df.to_dict("records")

In [64]:
for record in record_list:
    try:
        es.index(index="movies", document=record)
    except Exception as e:
        print(e)

BadRequestError(400, 'document_parsing_exception', '[1:512] failed to parse: [1:527] Non-standard token \'NaN\': enable `JsonReadFeature.ALLOW_NON_NUMERIC_NUMBERS` to allow\n at [Source: (byte[])"{"title":"The China Lake Murders","year":1990,"cast":["Tom Skerritt","Michael Parks"],"genres":["Crime","Drama"],"href":"The_China_Lake_Murders","extract":"The China Lake Murders is a 1990 television film starring Tom Skerritt. It is about a small desert town that experiences a series of murders. The film was rated PG-13 and first aired on the USA Network and for many years held the record for the highest rated basic cable film.\\nIt is an adaptation of the 1983 short film \\"China Lake\\" by Rober"[truncated 16276 bytes]; line: 1, column: 527]')
BadRequestError(400, 'document_parsing_exception', '[1:585] failed to parse: [1:600] Non-standard token \'NaN\': enable `JsonReadFeature.ALLOW_NON_NUMERIC_NUMBERS` to allow\n at [Source: (byte[])"{"title":"Coins in the Fountain","year":1990,"cast":["L

In [65]:
es.count(index="movies")

ObjectApiResponse({'count': 11750, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})